### note

In this note, we will investigate whether specialization on finer labels improve single model and logifold performances. We expect that finer labels doesn't helpful to single model performances, but may improve logifold performance. 

- Get finer labels
- Specialize with the finer labels each committee member. For comparison, we specialize with original label each committee member.

Above steps are done in the previous note:[`Make_New_label_using_adv_attacks.ipynb`](./Make_New_label_using_adv_attacks.ipynb).

-------

Then 
1. compare baseline models and specialized model with new labels. The specialized models answer in $0,\ldots, 19$, hence we subtract 10 from answers greater or equal to 10. 
2. compare baseline logifold and specialized models. For the latter, we may need 'filter' and 'experts'. For now, we simply put them together without filter nor experts.
- And get filter and experts from the Committee. Filter distinguish if it's 'dangerous' or not. Experts will be tuned on the 'dangerous' samples. Since training sample has very little dangerous Original and CWL2 samples (about one or two), we train them over the 'largely' perturbed data.


In [ ]:
# import necessary libraries and set experiments.

# load necessary libraries
import os, glob
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from keras.models import load_model
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import Model
from scipy.stats import entropy
import math
loaded_models = {}
# load additional models from another folder
# assuming these models are also saved in Keras format with '.keras' extension
# Say 'SB-type', where 'SB' stands for 'Specialized Base'.
folder = 'CIFAR10models/more_tunned/'
pattern = os.path.join(folder, '*_more_specialized*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models.update({os.path.basename(f): load_model(f) for f in file_list})

label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

(x, y), (x_test, y_test) = cifar10.load_data()
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, 
                                              random_state=42) # random state has been always 42.
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)
y_test_categorical_20 = to_categorical(y_test,20)

# Set the predetermined random seed
rng = np.random.default_rng(seed=42)

# Define ranges
range1 = [range(8), range(4), range(4), range(4)]
range2 = [range(8), range(4), [0], range(4)]

# Sample one tuple from each
tuple1 = tuple(rng.choice(r) for r in range1)

print("Comm 1:", tuple1)
comm_SB_1 = []
for file_name, model in loaded_models.items():
    # Extract model name from the file name
    base = file_name.replace('.keras','')
    parts = base.split('_')
    if parts[0] == 'n':
        model_name = label_dict[(int(parts[1]),int(parts[2][-1]))] + '_B_'+parts[-1][-1]
    else:
        model_name = parts[0] + '_SB_' + parts[-1]
    # print(f"Model: {model_name}")
    parts = model_name.split('_')
    if parts[0] == 'ResNet20v1':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[0]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet20v2':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[1]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet56v1':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[2]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet56v2':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[3]:
                comm_SB_1.append((model_name,model))
                
print('comm_SB_1', comm_SB_1)


# As SB model shows better performance on the certain part, let use SB type models to analyze the behavior on the certain part against adversarial examples.

# Load the adversarial examples

## set all files.
## Possibly there is no merged testing dataset, or validation dataset.
## We only look for testing dataset.
import re

pattern1 = re.compile(r'_\d+to\d+_')
pattern2 = re.compile(r'test')

folder = './adversarial_examples/gen_by_ResNet'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and not pattern1.search(fname) and not pattern2.search(fname)
]


## load all the training and validation dataset

loaded_training_dataset = {}
loaded_validation_dataset = {}
import re
for f in filtered_files:
    
    base = f.replace('.keras.npy','').replace('.npy','')
    parts = base.split('_')
    '''
    cwl2_x_tr_untargeted
    cwl2_x_v_untargeted.npy
    pgd_0.376_x_target_to_ll.npy
    pgd_0.376_x_target_to_second.npy
    pgd_0.376_x_untarget.npy
    pgd_0.376_x_val_target_to_ll.npy
    pgd_0.376_x_val_target_to_second.npy
    pgd_0.376_x_val_untarget.npy
    '''
    
    attack_type = parts[0]
    if attack_type == 'cwl2':
        direction = 'untargeted'
        if parts[2] == 'tr':
            train_or_val = 'train'
        else:
            train_or_val = 'val'
    else:
        attack_type = 'PGD by ResNet56v1'
        direction = parts[-1]
        if direction == 'll':
            direction = 'targeted to least'
        elif direction =='second':
            direction = 'targeted to 2nd'
        else:
            direction = 'untargeted'
        if parts[3] == 'val':
            train_or_val = 'val'
        else:
            train_or_val = 'train'
    key = (attack_type, direction, train_or_val)
    if train_or_val == 'train':
        loaded_training_dataset[key] = np.load(os.path.join(folder, f))
    else:
        loaded_validation_dataset[key] = np.load(os.path.join(folder, f))

print('-'*50, 'keys for adv samples', '-'*50)
for k in loaded_training_dataset.keys():
    print(k)
    
for k in loaded_validation_dataset.keys():
    print(k)
    
    
def get_entropy_array(committee : list[tuple[str,Model]], sample : np.ndarray): 
    '''
    param committee: list of tuple (member, model)
    param sample: numpy array of (n,) shape
    
    return tuple of dict, numpy array of (n,) shape
    dictionary : committee member -> list of [answer to the sample, certainty to the sample]
    '''
    comm_pred = {}
    for member, model in committee:
        p = model.predict(sample,verbose = 0)
        comm_pred[member] = [
                                np.argmax(p,axis=-1),
                                np.max(p,axis=-1)
                                ]
    stacked = np.stack([member_pred[0] for member_pred in comm_pred.values()], axis=1)
    counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=1, arr=stacked)
    probs = counts / counts.sum(axis=1, keepdims=True)
    disagreements = entropy(probs, axis=1, base = 10)
    
    return disagreements

committee = comm_SB_1

## CWL2 train and val
cwl2_train = loaded_training_dataset[('cwl2', 'untargeted', 'train')]
cwl2_val = loaded_validation_dataset[('cwl2', 'untargeted', 'val')]
entropy_cwl2_train = np.load('./entropies/entropy_SB1_cwl2_untargeted_train.npy')
entropy_cwl2_val = np.load('./entropies/entropy_SB1_cwl2_untargeted_val.npy')
cwl2_new_label_train = np.load('./new_labels/SB1/new_label_cwl2_untargeted_train.npy')
cwl2_new_label_val= np.load('./new_labels/SB1/new_label_cwl2_untargeted_val.npy')
cwl2_new_label_test= np.load('./new_labels/SB1/new_label_cwl2_untargeted_test.npy')
## PGD untargeted train and val
PGD_train = loaded_training_dataset[('PGD by ResNet56v1', 'untargeted', 'train')]
PGD_val = loaded_validation_dataset[('PGD by ResNet56v1', 'untargeted', 'val')]
entropy_PGD_untargeted_train = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_untargeted_train.npy')
entropy_PGD_untargeted_val = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_untargeted_val.npy')
PGD_untargeted_new_label_train = np.load('./new_labels/SB1/new_label_PGD_untargeted_train.npy')
PGD_untargeted_new_label_val = np.load('./new_labels/SB1/new_label_PGD_untargeted_val.npy')
PGD_untargeted_new_label_test = np.load('./new_labels/SB1/new_label_PGD_untargeted_test.npy')
## PGD targeted train and val
PGD_targeted_train = loaded_training_dataset[('PGD by ResNet56v1', 'targeted to least', 'train')]
PGD_targeted_val = loaded_validation_dataset[('PGD by ResNet56v1', 'targeted to least', 'val')]
entropy_PGD_targeted_train = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_targeted to least_train.npy')
entropy_PGD_targeted_val = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_targeted to least_val.npy')
PGD_targeted_new_label_train = np.load('./new_labels/SB1/new_label_PGD_targeted_train.npy')
PGD_targeted_new_label_val = np.load('./new_labels/SB1/new_label_PGD_targeted_val.npy')
PGD_targeted_new_label_test = np.load('./new_labels/SB1/new_label_PGD_targeted_test.npy')

def committee_answers(committee, sample, entropy, mask = None):
    if mask is None:
        # Create a mask for non-zero entropy values
        mask = entropy != 0

    # Prepare data for the table
    table_data = {
        'Original Label': y_train[:sample.shape[0]][mask].reshape(-1),
        'Entropy': np.round(entropy[mask], 3)
    }

    # Loop through the committee to get predictions
    for member, model in committee:
        # Predictions on CWL2 adversarial data
        p  = model.predict(sample[mask], verbose=0)
        table_data[f'{member[-1]}p'] = np.argmax(p , axis=-1)
        table_data[f'{member[-1]}c'] = np.round(np.max(p , axis=-1), 3)

    # Create and display the DataFrame
    df = pd.DataFrame(table_data)
    return df

def make_new_label(y_sample :np.ndarray, 
                   entropy_array : np.ndarray, 
                   entropy_thresholds : list[float] | float = 0.4, 
                   num_classes : int = 10):
    size = entropy_array.shape[0]
    y_sample_copy = y_sample.copy()[:size]
    new_label_cnt = 1
    if isinstance(entropy_thresholds,float):
        mask = entropy_array > entropy_thresholds
        y_sample_copy[mask] = y_sample_copy[mask]+num_classes*new_label_cnt
        return y_sample_copy, new_label_cnt
    elif len(entropy_thresholds) == 1:
        mask = entropy_array > entropy_thresholds[0]
        y_sample_copy[mask] = y_sample_copy[mask]+num_classes*new_label_cnt
        return y_sample_copy, new_label_cnt
    else:
        prev_th = entropy_thresholds[0]
        for curr_th in entropy_thresholds[1:]:
            mask = np.prod([entropy_array>prev_th,entropy_array<=curr_th],
                     axis=0)==1
            y_sample_copy[mask] = y_sample_copy[mask]+num_classes*new_label_cnt
            new_label_cnt += 1
        return y_sample_copy, new_label_cnt


In [ ]:
baseline_models = {}
folder = './new_labels/specialists/SB1/'
pattern = os.path.join(folder, '*_baseline.keras')
file_list = sorted(glob.glob(pattern))
baseline_models.update({os.path.basename(f): load_model(f) for f in file_list})

newly_labeled_models = {}
folder = './new_labels/specialists/SB1/'
pattern = os.path.join(folder, '*_new_label.keras')
file_list = sorted(glob.glob(pattern))
newly_labeled_models.update({os.path.basename(f): load_model(f) for f in file_list})

In [ ]:
cwl2_test = np.load('./adversarial_examples/gen_by_ResNet/cwl2_x_test_untargeted.npy')
PGD_untargeted_test = np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_untarget_gen_by_n_9_v1_cifar10.keras.npy')
PGD_targeted_test = np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_target_to_ll_gen_by_n_9_v1_cifar10.keras.npy')

table_data = {
        'Testing dataset': ['Original','CWL2','PGD untargeted','PGD targeted'],
    '20v1_Baseline':[0,0,0,0],
    '20v1_new_label':[0,0,0,0],
    '20v2_Baseline':[0,0,0,0],
    '20v2_new_label':[0,0,0,0],
    '56v1_Baseline':[0,0,0,0],
    '56v1_new_label':[0,0,0,0],
    '56v2_Baseline':[0,0,0,0],
    '56v2_new_label':[0,0,0,0],
    }

for model_name, baseline_model in baseline_models.items():
    base = model_name.replace('.keras','')
    parts = base.split('_')
    parts.pop()
    adv_type = parts[-1]
    table_values = [0,0,0,0]
    if adv_type == 'cwl2':
        testing_x = cwl2_test
        idx = 1
    elif adv_type == 'untargeted':
        adv_type = 'PGD untargeted'
        idx = 2
        testing_x = PGD_untargeted_test
    elif adv_type == 'targeted':
        adv_type = 'PGD targeted'
        idx = 3
        testing_x = PGD_targeted_test
    comparison_model = '_'.join(parts)+'_new_label.keras'
#     print(f"Model: {parts[1]+' '+ adv_type} ---- Baseline ----- Comparison")
    comparison_model = newly_labeled_models.get(comparison_model)
    
    pred_on_original_base = baseline_model.predict(x_test,verbose=0)
    pred_on_original_comparison = comparison_model.predict(x_test,verbose=0)
    a_base = np.argmax(pred_on_original_base,axis=-1)
    a_comp = np.argmax(pred_on_original_comparison,axis=-1)
    a_comp[a_comp>9] = a_comp[a_comp>9]-10
    acc_base = np.mean(a_base == y_test.reshape(-1))
    acc_comp = np.mean(a_comp == y_test.reshape(-1))
#     print(f"original test set: {acc_base:.4f} , {acc_comp:.4f} ")
    table_data[parts[1][-4:]+'_Baseline'][0] = acc_base
    table_data[parts[1][-4:]+'_new_label'][0] = acc_comp
    pred_on_testing_base = baseline_model.predict(testing_x,verbose=0)
    pred_on_testing_comparison = comparison_model.predict(testing_x,verbose=0)
    a_base = np.argmax(pred_on_testing_base,axis=-1)
    a_comp = np.argmax(pred_on_testing_comparison,axis=-1)
    acc_base = np.mean(a_base == y_test.reshape(-1))
    a_comp[a_comp>9] = a_comp[a_comp>9]-10
    acc_comp = np.mean(a_comp == y_test.reshape(-1))
#     print(f"{adv_type} test set: {acc_base:.4f} , {acc_comp:.4f}")
    table_data[parts[1][-4:]+'_Baseline'][idx] = acc_base
    table_data[parts[1][-4:]+'_new_label'][idx] = acc_comp
df = pd.DataFrame(table_data)
df

In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))
from logifold_modules.logifoldv1_4 import Logifold

2025-08-14 22:41:31.500459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
baseline_logifold = Logifold(10, name = 'Baseline_',
                             x_tr = x_train, y_tr = y_train_categorical_10,
                             x_v = x_val, y_v = y_val_categorical_10,
                             path = os.path.abspath('../')+'/logifolds/exp_new_labels/baseline/')

comparison_logifold = Logifold(20, name = 'comparison_',
                             x_tr = x_train, y_tr = y_train_categorical_10,
                             x_v = x_val, y_v = y_val_categorical_10,
                             path = os.path.abspath('../')+'/logifolds/exp_new_labels/comparison/')

In [ ]:
for model_name, baseline_model in baseline_models.items():
    # model name : specialist_{member}_{adv_name}_baseline.keras
    base = model_name.replace('.keras','')
    parts = base.split('_')
    parts[-2] = adv_type
    if adv_type == 'cwl2':
        first_idx = 1
    elif adv_type == 'untargeted':
        first_idx = 2
    elif adv_type == 'targeted':
        first_idx = 3
    # parts[1] = ResNet##v#
    second_idx = int(parts[1][5:7]+ parts[1][-1])
    key = (first_idx, second_idx)
    # second index: 201, 202, 561, 562
    baseline_logifold.add(baseline_model, key)
    
for model_name, comparison_model in newly_labeled_models.items():
    # model name : specialist_{member}_{adv_name}_new_label.keras
    base = model_name.replace('.keras','')
    parts = base.split('_')
    parts[-3] = adv_type
    if adv_type == 'cwl2':
        first_idx = 1
    elif adv_type == 'PGD untargeted':
        first_idx = 2
    elif adv_type == 'PGD targeted':
        first_idx = 3
    # parts[1] = ResNet##v#
    second_idx = int(parts[1][5:7]+ parts[1][-1])
    key = (first_idx, second_idx)
    # second index: 201, 202, 561, 562
    comparison_logifold.add(comparison_model, key)

In [ ]:
'''
cwl2_new_label_test
PGD_untargeted_new_label_test
PGD_targeted_new_label_test
'''
# new label original test dataset
entropy_array = get_entropy_array(committee,x_test)
np.save('./new_labels/SB1/new_label_original_test.npy',
       make_new_label(y_test, entropy_array)[0]
        )

original_new_label_test = np.load('./new_labels/SB1/new_label_original_test.npy')
testing_datasets_no_new_label = [y_test_categorical_10,
                                 to_categorical(cwl2_test,10),
                                 to_categorical(PGD_untargeted_test,10),
                                 to_categorical(PGD_targeted_test,10)
                                ]
testing_datasets_new_label = [to_categorical(original_new_label_test,20),
                   to_categorical(cwl2_new_label_test,20),
                   to_categorical(PGD_untargeted_new_label_test,20),
                   to_categorical(PGD_targeted_new_label_test,20)
                  ]

Computation fuzzy domain follows the target label. For instance, tager labels are 0 - 9, then we sample from original dataset.


Compare with separation by method.. (by PGD or Step size) 

Logifold1 : label by methods

Logifold2 : label by entropies

Union of original and PGD

-> by original and PGD

-> by entropy

in terms of filter first